# Processing Glasses Dataset with Hunyuan3D-2

This notebook demonstrates how to process a dataset of glasses images and generate 3D models using the adapted Hunyuan3D-2 model.

## Setup

First, let's set up the environment and clone the repository.

In [ ]:
# Clone the repository
!git clone https://github.com/amin8452/Pfa-3D.git
%cd Pfa-3D

In [ ]:
# Install dependencies
!pip install torch torchvision
!pip install trimesh numpy scipy matplotlib opencv-python pyyaml tqdm tensorboard plotly requests
!pip install safetensors

# Install PyTorch3D (this can be complex, so we use a pre-built version for Colab)
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

## Download and Adapt the Hunyuan3D-2 Model

Now, let's download and adapt the Hunyuan3D-2 model.

In [ ]:
# Clone the Hunyuan3D-2 repository
!python scripts/download_pretrained.py --model repo

# Download pre-trained models from Hugging Face
!python scripts/download_pretrained.py --model all

# Adapt the model for glasses reconstruction
# Note: Use the correct directory name where the repository was cloned
!python scripts/adapt_hunyuan_model.py --hunyuan_dir Hunyuan3D-2

## Upload Your Dataset

Now, let's upload your dataset of glasses images.

In [ ]:
# Create data directory
!mkdir -p data/glasses/images

# Upload images
from google.colab import files
import os
import zipfile

# Option 1: Upload a zip file containing all images
print("Please upload a zip file containing your glasses images...")
uploaded = files.upload()

# Extract the zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('data/glasses')
        print(f"Extracted {filename} to data/glasses")
    else:
        # If it's not a zip file, assume it's an image and save it
        with open(os.path.join('data/glasses/images', filename), 'wb') as f:
            f.write(uploaded[filename])
        print(f"Saved {filename} to data/glasses/images")

In [ ]:
# Option 2: Upload individual images
# You can run this cell multiple times to upload more images
print("Please upload individual glasses images...")
uploaded = files.upload()

for filename, content in uploaded.items():
    with open(os.path.join('data/glasses/images', filename), 'wb') as f:
        f.write(content)
    print(f"Saved {filename} to data/glasses/images")

## Check the Dataset

Let's check the dataset to make sure it's properly loaded.

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Path to the dataset
dataset_path = 'data/glasses'

# Find all image files
image_files = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
            image_files.append(os.path.join(root, file))

print(f"Found {len(image_files)} images in {dataset_path}")

# Display a few sample images
if image_files:
    plt.figure(figsize=(15, 10))
    for i, image_path in enumerate(image_files[:5]):
        plt.subplot(1, 5, i+1)
        img = Image.open(image_path)
        plt.imshow(np.array(img))
        plt.title(os.path.basename(image_path))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

## Process the Dataset

Now, let's process the dataset and generate 3D models.

In [ ]:
# Process the dataset
!python scripts/process_dataset.py --dataset_path data/glasses --output_dir results/meshes --checkpoint checkpoints/hunyuan3d_base.safetensors

## Visualize the Results

Let's visualize some of the generated 3D models.

In [ ]:
import os
import trimesh
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Path to the generated meshes
meshes_dir = 'results/meshes'

# Find all mesh files
mesh_files = [f for f in os.listdir(meshes_dir) if f.endswith('.obj')]

print(f"Found {len(mesh_files)} meshes in {meshes_dir}")

# Display a few sample meshes
if mesh_files:
    plt.figure(figsize=(15, 10))
    for i, mesh_file in enumerate(mesh_files[:3]):
        mesh_path = os.path.join(meshes_dir, mesh_file)
        mesh = trimesh.load(mesh_path)
        
        # Get the vertices
        vertices = np.array(mesh.vertices)
        
        # Create a 3D plot
        ax = plt.subplot(1, 3, i+1, projection='3d')
        ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], s=1)
        ax.set_title(mesh_file)
        ax.axis('off')
        
        # Set equal aspect ratio
        max_range = np.max([
            np.ptp(vertices[:, 0]),
            np.ptp(vertices[:, 1]),
            np.ptp(vertices[:, 2])
        ])
        mid_x = np.mean([np.min(vertices[:, 0]), np.max(vertices[:, 0])])
        mid_y = np.mean([np.min(vertices[:, 1]), np.max(vertices[:, 1])])
        mid_z = np.mean([np.min(vertices[:, 2]), np.max(vertices[:, 2])])
        ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
        ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
        ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)
    
    plt.tight_layout()
    plt.show()

## Download the Results

Finally, let's download the generated 3D models.

In [ ]:
# Compress the results into a zip file
!zip -r results.zip results/meshes

# Download the zip file
from google.colab import files
files.download('results.zip')